In [1]:
import numpy as np

In [2]:
def identity(x):
    return x
def d_identity_dx(x):
    return 1

In [3]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
def d_sigmoid_dx(x):
    return sigmoid(x)*(1-sigmoid(x))
sigmoid(-1000), sigmoid(0), sigmoid(1000)

/var/folders/hd/kdltsywn53jbp7lv3g1hld3w0000gn/T/ipykernel_51740/3566510964.py:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


(0.0, 0.5, 1.0)

In [44]:
class DenseLayer:
    
    def __init__(self, activation_function, dactivation_function):
        self.activation_function=activation_function
        self.dactivation_function=dactivation_function
    
    def foward_pass(self, x):
        x = np.array(x)
        # print(self.w.shape, x.shape)
        s = np.matmul(self.w, x)
        # print(s)
        return self.activation_function(s)
    
    def backward_pass(self, x):
        x = np.array(x)
        
        print(x.shape)
        
        s = np.matmul(self.w, x)
        
        print(s.shape)
        
        dactivation = np.array(self.dactivation_function(s))
        
        print(dactivation.shape)
        
        # d/dx
        dsdx = self.w.T
        
        print(dactivation.shape, dsdx.shape)
        
        dx = np.multiply(dactivation, dsdx)
        
        # d/dw
        dsdw = x
        
        print(dactivation.shape, dsdw.shape)
        
        dw = np.multiply(dactivation, dsdw)
        
        return dsdx, dsdw
    

In [45]:

z1 = DenseLayer(sigmoid, d_sigmoid_dx)
z2 = DenseLayer(sigmoid, d_sigmoid_dx)
y =  DenseLayer(identity, d_identity_dx)


In [46]:
z1.w = np.array(
    [
        [-1, -2, -3],
        [ 1,  2,  3]
    ]
)
z1.w

array([[-1, -2, -3],
       [ 1,  2,  3]])

In [47]:
z2.w = np.array(
    [
        [-1, -2, -3],
        [ 1,  2,  3]
    ]
)
z2.w

array([[-1, -2, -3],
       [ 1,  2,  3]])

In [48]:
y.w = np.array(
    [
        [ -1,  2, -1.5]
    ]
)
y.w

array([[-1. ,  2. , -1.5]])

In [49]:
def model(x):
    Z1 = [1, *z1.foward_pass(x)]
    Z2 = [1, *z2.foward_pass(Z1)]
    Y =  y.foward_pass(Z2)
    return Y

In [50]:
model(np.array([1,1,1]))

array([-2.43689523])

In [51]:
def backprop(x, y_true):
    Z1 = np.array([1, *z1.foward_pass(x)])
    Z2 = np.array([1, *z2.foward_pass(Z1)])
    Y =  y.foward_pass(Z2)
    print(Y)
    L = (1/2) * (Y-y_true)**2
    
    dLdy = Y-y_true
    
    dydz2, dydw3 = y.backward_pass(Z2)
    
    dLdw3 = dLdy*dydw3
    
    print(Z1.shape, z2.w.shape)
    
    dydz1, dydw2 = z2.backward_pass(Z1)
    
    dLdw2 = dLdy * dydz2 * dz2dw2
    
    print(dLdw2)
    return Y
backprop(np.array([1,1,1]), np.array(1))

[-2.43689523]
(3,)
(1,)
()
() (3, 1)
() (3,)
(3,) (2, 3)
(3,)
(2,)
(2,)
(2,) (3, 2)
(2,) (3,)


ValueError: operands could not be broadcast together with shapes (2,) (3,) 